In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Load the datasets
temperature_data = pd.read_csv("../data/NSW/aggregated_temperature_data.csv")
humidity_data = pd.read_csv("../data/NSW/aggregated_humidity_data.csv")
wind_speed_data = pd.read_csv("../data/NSW/aggregated_windspeed_data.csv")
wind_direction_data = pd.read_csv("../data/NSW/aggregated_wind_direction_data.csv")
solar_radiation_data = pd.read_csv("../data/NSW/aggregated_solar_radiation_data.csv")
population_forecast_data = pd.read_csv("../data/NSW/PopulationForecastNSW.csv")
energy_demand_data = pd.read_csv("../data/NSW/totaldemand_nsw.csv")
enso_data = pd.read_csv("../data/NSW/daily_enso.csv")
soi_data = pd.read_csv("../data/NSW/soi_monthly.csv")

# 2. Preprocess the datasets
def preprocess_datetime(df, date_col='Date', time_col='Time'):
    # Convert the 'Date' column to datetime format
    df[date_col] = pd.to_datetime(df[date_col], dayfirst=True, errors='coerce')
    
    # Ensure the 'Time' column is a string before concatenating with the 'Date'
    if time_col in df.columns:
        df[time_col] = df[time_col].astype(str)
        df['DateTime'] = pd.to_datetime(df[date_col].dt.strftime('%Y-%m-%d') + ' ' + df[time_col], errors='coerce')
    else:
        df['DateTime'] = df[date_col]
    
    # Drop any rows with invalid DateTime values
    df = df.dropna(subset=['DateTime'])
    
    return df

# Apply the preprocessing function to all datasets (adjusting for column names)
temperature_data = preprocess_datetime(temperature_data, 'Date', 'Time')
humidity_data = preprocess_datetime(humidity_data, 'Date', 'Time')
wind_speed_data = preprocess_datetime(wind_speed_data, 'Date', 'Time')
wind_direction_data = preprocess_datetime(wind_direction_data, 'Date', 'Time')
solar_radiation_data = preprocess_datetime(solar_radiation_data, 'Date', 'Time')

# For enso_data, the date column is 'DATE' in MM/DD/YYYY format
enso_data['DATE'] = pd.to_datetime(enso_data['DATE'], format='%m/%d/%Y', errors='coerce')

# For soi_data, the 'yearmonth' needs to be converted to a proper date
soi_data['yearmonth'] = soi_data['yearmonth'].astype(str) + '01'  # Append '01' to create a valid date
soi_data['DateTime'] = pd.to_datetime(soi_data['yearmonth'], format='%Y%m%d')

# 3. Merge datasets on the common DateTime column
merged_data = temperature_data[['DateTime', 'mean_temp']].merge(humidity_data[['DateTime', 'mean_humidity']], on='DateTime', how='outer')
merged_data = merged_data.merge(wind_speed_data[['DateTime', 'mean_windspeed']], on='DateTime', how='outer')
merged_data = merged_data.merge(wind_direction_data[['DateTime', 'mean_wind_direction']], on='DateTime', how='outer')
merged_data = merged_data.merge(solar_radiation_data[['DateTime', 'mean_solar_radiation']], on='DateTime', how='outer')
merged_data = merged_data.merge(enso_data[['DATE', 'SOI']], left_on='DateTime', right_on='DATE', how='outer')
merged_data = merged_data.merge(soi_data[['DateTime', 'soi']], on='DateTime', how='outer')

# 4. Merge with population forecast data (for yearly predictions)
population_forecast_data['Year'] = population_forecast_data['Year'].astype(int)
population_forecast_data = population_forecast_data[['Year', 'Medium_Series']]
merged_data['Year'] = merged_data['DateTime'].dt.year
merged_data = merged_data.merge(population_forecast_data, on='Year', how='left')

# 5. Merge with energy demand data
energy_demand_data['DATETIME'] = pd.to_datetime(energy_demand_data['DATETIME'], errors='coerce')
merged_data = merged_data.merge(energy_demand_data[['DATETIME', 'TOTALDEMAND']], left_on='DateTime', right_on='DATETIME', how='inner')

# 6. Handle missing values
merged_data.fillna(method='ffill', inplace=True)

# 7. Feature Engineering - create time-based features
merged_data['Hour'] = merged_data['DateTime'].dt.hour
merged_data['Day'] = merged_data['DateTime'].dt.day
merged_data['Month'] = merged_data['DateTime'].dt.month
merged_data['Week'] = merged_data['DateTime'].dt.isocalendar().week

# 8. Define features (X) and target (y)
features = ['mean_temp', 'mean_humidity', 'mean_windspeed', 'mean_wind_direction', 'mean_solar_radiation', 'SOI', 'Medium_Series', 'Hour', 'Day', 'Month', 'Week']
X = merged_data[features]
y = merged_data['TOTALDEMAND']

# 9. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 10. Fit Linear Regression Model
model = LinearRegression()
model.fit(X_train, y_train)

# 11. Make predictions
y_pred = model.predict(X_test)

# 12. Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'MSE: {mse}')
print(f'R-squared: {r2}')

# 13. Plot results
plt.figure(figsize=(10, 6))
plt.plot(y_test.values[:100], label='Actual')
plt.plot(y_pred[:100], label='Predicted')
plt.title('Actual vs Predicted Energy Demand (Hourly)')
plt.legend()
plt.show()

# 14. Aggregation for daily, weekly, monthly, and yearly predictions
# Daily
daily_data = merged_data.resample('D', on='DateTime').mean()
X_daily = daily_data[features]
y_daily = daily_data['TOTALDEMAND']
y_pred_daily = model.predict(X_daily)

# Weekly
weekly_data = merged_data.resample('W', on='DateTime').mean()
X_weekly = weekly_data[features]
y_weekly = weekly_data['TOTALDEMAND']
y_pred_weekly = model.predict(X_weekly)

# Monthly
monthly_data = merged_data.resample('M', on='DateTime').mean()
X_monthly = monthly_data[features]
y_monthly = monthly_data['TOTALDEMAND']
y_pred_monthly = model.predict(X_monthly)

# Yearly
yearly_data = merged_data.resample('Y', on='DateTime').mean()
X_yearly = yearly_data[features]
y_yearly = yearly_data['TOTALDEMAND']
y_pred_yearly = model.predict(X_yearly)

# 15. Plot daily, weekly, monthly, and yearly predictions
plt.figure(figsize=(10, 6))
plt.plot(y_daily[:100], label='Actual Daily')
plt.plot(y_pred_daily[:100], label='Predicted Daily')
plt.title('Daily Energy Demand')
plt.legend()
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(y_weekly[:20], label='Actual Weekly')
plt.plot(y_pred_weekly[:20], label='Predicted Weekly')
plt.title('Weekly Energy Demand')
plt.legend()
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(y_monthly[:12], label='Actual Monthly')
plt.plot(y_pred_monthly[:12], label='Predicted Monthly')
plt.title('Monthly Energy Demand')
plt.legend()
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(y_yearly, label='Actual Yearly')
plt.plot(y_pred_yearly, label='Predicted Yearly')
plt.title('Yearly Energy Demand')
plt.legend()
plt.show()


MemoryError: Unable to allocate 9.01 GiB for an array with shape (8, 151210810) and data type float64

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Load the datasets
temperature_data = pd.read_csv("../data/NSW/aggregated_temperature_data.csv")
humidity_data = pd.read_csv("../data/NSW/aggregated_humidity_data.csv")
wind_speed_data = pd.read_csv("../data/NSW/aggregated_windspeed_data.csv")
wind_direction_data = pd.read_csv("../data/NSW/aggregated_wind_direction_data.csv")
solar_radiation_data = pd.read_csv("../data/NSW/aggregated_solar_radiation_data.csv")
population_forecast_data = pd.read_csv("../data/NSW/PopulationForecastNSW.csv")
energy_demand_data = pd.read_csv("../data/NSW/totaldemand_nsw.csv")
enso_data = pd.read_csv("../data/NSW/daily_enso.csv")
soi_data = pd.read_csv("../data/NSW/soi_monthly.csv")
# Check the column names of enso_data
print(enso_data.columns)

# Adjust the column name if necessary
if 'DATE' not in enso_data.columns:
    # Inspect the column names and find the correct one
    print("Column 'DATE' not found. Available columns:", enso_data.columns)

# If the correct column is different, update this line accordingly
enso_data['DATE'] = pd.to_datetime(enso_data['DATE'], format='%m/%d/%Y', errors='coerce')


Index(['DATE', 'SOI', 'SST_DIFF', 'enso'], dtype='object')
